In [ ]:
!pip3 install -U transformers datasets evaluate accelerate
!pip3 install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install peft bitsandbytes

In [ ]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, TaskType

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

dataset = load_dataset('chathuru/cicids2018')

model_name = "tiiuae/falcon-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    load_in_8bit=True,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)
model.config.use_cache = False

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=8,
    bias="none",
    task_type=TaskType.TOKEN_CLS
)
model.add_adapter(peft_config)

training_args = TrainingArguments(
    output_dir="cicids2018-falcon7b",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    fp16=True,
    optim="paged_adamw_32bit",
    logging_steps=1,
    gradient_accumulation_steps=4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
config = AutoConfig.from_pretrained("tiiuae/falcon-7b")

trainer.push_to_hub("cicids2018-falcon7b")
model.push_to_hub("cicids2018-falcon7b")
model.config.push_to_hub("cicids2018-falcon7b")

config.push_to_hub("cicids2018-falcon7b")

trainer.save_model("cicids2018-falcon7b")
tokenizer.save_pretrained("cicids2018-falcon7b")